<a href="https://colab.research.google.com/github/MHosseinHashemi/Face-Recognition-System/blob/main/Second_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [ ]:
!pip install facenet_pytorch

In [ ]:
# !pip install opencv-python

In [ ]:
# !pip install unrar

In [ ]:
from torch import optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import zipfile
import torch
import uuid
import time
import cv2
import os
import warnings
warnings.filterwarnings('ignore')

# Collect Data

In [ ]:
# root_dir = "/content/Train_Data"

# for i in range(300):
#   cap = cv2.VideoCapture(0)
#   print(f"Taking Picture number {i} ......")
#   time.sleep(5)
#   ret, frame = cap.read()
#   imgname = os.path.join(root_dir, f"_{i}.jpg")
#   cv2.imwrite(imgname, frame)
#   cv2.imshow('frame', frame)
#   time.sleep(3)

#   if cv2.waitKey(1) & 0xFF == ord('q'):
#     break

# cap.release()
# cv2.destroyAllWindows()

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# def take_photo(filename='/content/Train_Data/photo.jpg', quality=0.8):
  # root = "/content/Train_Data"
  # filename = input("Image named as ")
  # filename = os.path.join(root, filename + ".jpg")

def take_photo(filename, quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image

try:
  for index in range(111,500):
    print(f"Taking Picture number {index+1}/500 ......")
    root = "/content/Train_Data"
    filename = os.path.join(root, "Sample_" + str(index) + ".jpg")
    path = take_photo(filename)
    print('Saved to {}'.format(path))
    print("-"*50)
    time.sleep(2)

except Exception as err:
  print(str(err))

# Data Augmentation

In [ ]:
!unzip /content/dataset.zip -d /content/

Archive:  /content/dataset.zip
   creating: /content/dataset/
   creating: /content/dataset/Person1/
  inflating: /content/dataset/Person1/Sample_0.jpg  
  inflating: /content/dataset/Person1/Sample_0_BR_.jpg  
  inflating: /content/dataset/Person1/Sample_0_DA_.jpg  
  inflating: /content/dataset/Person1/Sample_0_HF_.jpg  
  inflating: /content/dataset/Person1/Sample_0_RO_.jpg  
  inflating: /content/dataset/Person1/Sample_1.jpg  
  inflating: /content/dataset/Person1/Sample_10.jpg  
  inflating: /content/dataset/Person1/Sample_100.jpg  
  inflating: /content/dataset/Person1/Sample_100_BR_.jpg  
  inflating: /content/dataset/Person1/Sample_100_DA_.jpg  
  inflating: /content/dataset/Person1/Sample_100_HF_.jpg  
  inflating: /content/dataset/Person1/Sample_100_RO_.jpg  
  inflating: /content/dataset/Person1/Sample_101.jpg  
  inflating: /content/dataset/Person1/Sample_101_BR_.jpg  
  inflating: /content/dataset/Person1/Sample_101_DA_.jpg  
  inflating: /content/dataset/Person1/Sample_10

In [ ]:
import Data_and_Label_Augmentation as DataGen

#### Initialize Image Custom Augmentation object
my_data = DataGen.Image_Custom_Augmentation(SP_intensity=False,  # Salt and Pepper Intensity
                                            RO_Key=15,           # Rotation Intensity
                                            Br_intensity=15,     # Brightness Intensity
                                            H_Key=True,          # Horizontal Flip
                                            V_Key=False,         # Vertical Flip
                                            HE_Key=False,        # Histogram Equalization
                                            Img_res= 512)        # Image Resolution

#### Generate augmented data
my_data.Generate_Data(input_path="/content/dataset/Person1", output_path="/content/dataset/Person1")

# Fine-Tune

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [ ]:
# rar_file = '/content/data.rar'

# # Create a directory to extract files to
# extract_dir = '/content/'
# os.makedirs(extract_dir, exist_ok=True)

# # Extract the RAR file
# !unrar x -r /content/data.rar /content/

In [ ]:
# !rm -R /content/data/_croppedPerson1
# !ls /content/data -a   #to make sure that the deletion has occurred

In [ ]:
# !zip -r /content/dataset.zip /content/dataset



> Error Handling:

A hidden .ipynb file causes error, so make sure to run the cells bellow with respect to the directory of your dataset



In [ ]:
!rm -R /content/dataset/.ipynb_checkpoints

rm: cannot remove '/content/dataset/.ipynb_checkpoints': No such file or directory


In [ ]:
!rm -R /content/dataset/Person1/.ipynb_checkpoints

rm: cannot remove '/content/dataset/Person1/.ipynb_checkpoints': No such file or directory


In [ ]:
# !unzip /content/dataset.zip -d /content/dataset

Archive:  /content/dataset.zip
   creating: /content/dataset/content/dataset/
   creating: /content/dataset/content/dataset/Person1/
  inflating: /content/dataset/content/dataset/Person1/Sample_128_BR_.jpg  
  inflating: /content/dataset/content/dataset/Person1/Sample_53_DA_.jpg  
  inflating: /content/dataset/content/dataset/Person1/Sample_17_DA_.jpg  
  inflating: /content/dataset/content/dataset/Person1/Sample_125_DA_.jpg  
  inflating: /content/dataset/content/dataset/Person1/Sample_132_HF_.jpg  
  inflating: /content/dataset/content/dataset/Person1/Sample_78_HF_.jpg  
  inflating: /content/dataset/content/dataset/Person1/Sample_103_BR_.jpg  
  inflating: /content/dataset/content/dataset/Person1/Sample_36_BR_.jpg  
  inflating: /content/dataset/content/dataset/Person1/Sample_94_DA_.jpg  
  inflating: /content/dataset/content/dataset/Person1/Sample_41_HF_.jpg  
  inflating: /content/dataset/content/dataset/Person1/Sample_23_HF_.jpg  
  inflating: /content/dataset/content/dataset/Per

In [ ]:
data_dir = '/content/dataset'

batch_size = 16
epochs = 20
workers = 0 if os.name == 'nt' else 8

In [ ]:
# D_gen = DataGen.Image_Custom_Augmentation(RO_Key=30, Br_intensity=30, H_Key=True, Img_res=512)

In [ ]:
# D_gen.Generate_Data(input_path="/content/data/Person1", output_path="/content/augmented_data/Person1")

100%|██████████| 50/50 [00:02<00:00, 23.42it/s]


In [ ]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [ ]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]

loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')

# Remove mtcnn to reduce GPU memory usage
del mtcnn

Batch 63 of 63

In [ ]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [ ]:
!rm -R /content/dataset_cropped/.ipynb_checkpoints

rm: cannot remove '/content/dataset_cropped/.ipynb_checkpoints': No such file or directory


In [ ]:
!rm -R /content/dataset_cropped/Person1/.ipynb_checkpoints

rm: cannot remove '/content/dataset_cropped/Person1/.ipynb_checkpoints': No such file or directory


In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)


trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

dataset = datasets.ImageFolder("/content/dataset_cropped", transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.9 * len(img_inds))]
val_inds = img_inds[int(0.9 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

In [ ]:
# loss_fn = torch.nn.CrossEntropyLoss()
loss_fn = torch.nn.BCEWithLogitsLoss()
# metrics = {
#     'fps': training.BatchTimer(),
#     'acc': training.accuracy
# }

In [ ]:
def compute_accuracy(y_pred, y_true):
    y_pred_binary = (y_pred > 0.5).float()
    correct = (y_pred_binary == y_true).sum().item()
    total = y_true.size(0)
    accuracy = correct / total
    return accuracy

In [ ]:
# Training loop
for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))

    # Training loop
    resnet.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    resnet.train()
    for x, y in train_loader:
        y = y.float().unsqueeze(1).to(device)
        y_pred = resnet(x.to(device))
        loss_batch = loss_fn(y_pred, y)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss_batch.backward()
        optimizer.step()

        # Loss Report
        train_loss += loss_batch.item() * x.size(0)
        y_pred_binary = (y_pred > 0.5).float()
        train_correct += (y_pred_binary == y).sum().item()
        train_total += y.size(0)

    train_loss /= train_total
    train_accuracy = train_correct / train_total

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Train Accuracy: {train_accuracy:.4f}")


    # Validation Loop
    resnet.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
      for x, y in val_loader:
          y = y.float().unsqueeze(1).to(device)
          y_pred = resnet(x.to(device))
          loss_batch = loss_fn(y_pred, y)

          val_loss += loss_batch.item() * x.size(0)
          y_pred_binary = (y_pred > 0.5).float()
          val_correct += (y_pred_binary == y).sum().item()
          val_total += y.size(0)

    val_loss /= val_total
    val_accuracy = val_correct / val_total
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print("="*100)

    scheduler.step(val_loss)




Epoch 1/20
Train Loss: 0.6519
Train Accuracy: 0.8833
Validation Loss: 0.3477
Validation Accuracy: 1.0000

Epoch 2/20
Train Loss: 0.2505
Train Accuracy: 0.9989
Validation Loss: 0.3226
Validation Accuracy: 0.9500

Epoch 3/20
Train Loss: 0.0638
Train Accuracy: 0.9989
Validation Loss: 0.0324
Validation Accuracy: 1.0000

Epoch 4/20
Train Loss: 0.0227
Train Accuracy: 1.0000
Validation Loss: 1.1499
Validation Accuracy: 0.9000

Epoch 5/20
Train Loss: 0.0223
Train Accuracy: 0.9989
Validation Loss: 0.0144
Validation Accuracy: 1.0000

Epoch 6/20
Train Loss: 0.0174
Train Accuracy: 0.9967
Validation Loss: 0.0083
Validation Accuracy: 1.0000

Epoch 7/20
Train Loss: 0.0093
Train Accuracy: 0.9989
Validation Loss: 0.0059
Validation Accuracy: 1.0000

Epoch 8/20
Train Loss: 0.0058
Train Accuracy: 0.9989
Validation Loss: 0.0041
Validation Accuracy: 1.0000

Epoch 9/20
Train Loss: 0.0041
Train Accuracy: 1.0000
Validation Loss: 0.0025
Validation Accuracy: 1.0000

Epoch 10/20
Train Loss: 0.0021
Train Accuracy

In [2]:
# torch.save(resnet.state_dict(), '/content/facenet_model_final.pth')